In [1]:
import sys 

!{sys.executable} -m pip install google-play-scraper
!{sys.executable} -m pip install tzlocal

     |████████████████████████████████| 52 kB 1.1 MB/s  eta 0:00:01
  Created wheel for google-play-scraper: filename=google_play_scraper-1.0.0-py3-none-any.whl size=24069 sha256=42d006cf910e2402151ae2297416ee102c3bd357b57a8e0c56c39447a22d1a26
  Stored in directory: /home/ec2-user/.cache/pip/wheels/5c/22/78/b708165d9158a7c96e65cb15e6ab57e537417f81db7afa68e4
Successfully built google-play-scraper


## Scrape Google Play Store reviews using the google_play_scraper library

Homepage: https://github.com/JoMingyu/google-play-scraper

---

In [1]:
import pandas as pd

# for scraping app info and reviews from Google Play
from google_play_scraper import app, Sort, reviews

# for pretty printing data structures
from pprint import pprint

# for keeping track of timing
import datetime as dt
from tzlocal import get_localzone

# for building in wait times
import random
import time

C:\Users\matthewkent.regalado\Anaconda3\envs\project-env\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\matthewkent.regalado\Anaconda3\envs\project-env\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\matthewkent.regalado\Anaconda3\envs\project-env\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [21]:
app_name = 'GCash'
app_id = 'com.globe.gcash.android'
    
# Get start time
start = dt.datetime.now(tz=get_localzone())
fmt= "%m/%d/%y - %T %p"    

# Print starting output for app
print('---'*20)
print('---'*20)    
print(f'***** {app_name} started at {start.strftime(fmt)}')
print()

# Empty list for storing reviews
app_reviews = []

# Number of reviews to scrape per batch
count = 200

# To keep track of how many batches have been completed
batch_num = 0


# Retrieve reviews (and continuation_token) with reviews function
rvws, token = reviews(
    app_id,           # found in app's url
    lang='en',        # defaults to 'en'
    country='ph',     # defaults to 'us'
    sort=Sort.NEWEST, # start with most recent
    count=count,       # batch size
    filter_score_with=5 # defaults to None(means all score)
)


# Add the list of review dicts to overall list
app_reviews.extend(rvws)

# Increase batch count by one
batch_num +=1 
print(f'Batch {batch_num} completed.')

# convert list of review dictionaries to a dataframe
gps_df = pd.DataFrame(app_reviews)

# Wait 20 to 30 seconds to start next batch
time.sleep(random.randint(3, 5))

# Append review IDs to list prior to starting next batch
pre_review_ids = []
for rvw in app_reviews:
    pre_review_ids.append(rvw['reviewId'])

# Loop through at most max number of batches
for batch in range(1000):
    rvws, token = reviews( # store continuation_token
        app_id,
        lang='en',
        country='ph',
        sort=Sort.NEWEST,
        count=count,
        continuation_token=token, # using token obtained from previous batch
        filter_score_with=5 # defaults to None(means all score)
    )

    # Append unique review IDs from current batch to new list
    new_review_ids = []
    for r in rvws:
        new_review_ids.append(r['reviewId'])

    # Add the list of review dicts to main app_reviews list
    app_reviews.extend(rvws)

    # Increase batch count by one
    batch_num +=1

    # Break loop and stop scraping for current app if most recent batch
      # did not add any unique reviews
    all_review_ids = pre_review_ids + new_review_ids
    if len(set(pre_review_ids)) == len(set(all_review_ids)):
        print(f'No reviews left to scrape. Completed {batch_num} batches.\n')
        break

    # all_review_ids becomes pre_review_ids to check against 
      # for next batch
    pre_review_ids = all_review_ids

    # At every 100th batch
    if batch_num%100==0:

        # print update on number of batches
        print(f'Batch {batch_num} completed.')

        # print update about num reviews inserted
        store_time = dt.datetime.now(tz=get_localzone())
        print(f"""
        Successfully inserted {len(app_reviews)} {app_name} 
        reviews into collection at {store_time.strftime(fmt)}.\n
        """)

#         # empty our list for next round of 100 batches
#         app_reviews = []

    # Wait 20 to 30 seconds to start next batch
    time.sleep(random.randint(3, 5))

# Print update when max number of batches has been reached
  # OR when last batch didn't add any unique reviews
print(f'Done scraping {app_name}.')
print(f'Scraped a total of {len(set(pre_review_ids))} unique reviews.\n')

# Append new reviews to main dataframe
gps_df = gps_df.append(app_reviews, ignore_index=True)

# Get end time
end = dt.datetime.now(tz=get_localzone())

# Print ending output for app
print(f"""
Successfully inserted all {app_name} reviews into collection
at {end.strftime(fmt)}.\n
""")
print(f'Time elapsed for {app_name}: {end-start}')
print('---'*20)
print('---'*20)
print('\n')

------------------------------------------------------------
------------------------------------------------------------
***** GCash started at 06/04/21 - 13:45:59 PM

Batch 1 completed.
Batch 100 completed.

        Successfully inserted 20000 GCash 
        reviews into collection at 06/04/21 - 13:53:33 PM.

        
Batch 200 completed.

        Successfully inserted 40000 GCash 
        reviews into collection at 06/04/21 - 14:00:43 PM.

        
Batch 300 completed.

        Successfully inserted 60000 GCash 
        reviews into collection at 06/04/21 - 14:08:15 PM.

        
Batch 400 completed.

        Successfully inserted 80000 GCash 
        reviews into collection at 06/04/21 - 14:16:22 PM.

        
Batch 500 completed.

        Successfully inserted 100000 GCash 
        reviews into collection at 06/04/21 - 14:24:24 PM.

        
Batch 600 completed.

        Successfully inserted 120000 GCash 
        reviews into collection at 06/04/21 - 14:32:19 PM.

        
No rev

In [22]:
gps_df.sort_values(by=['at'])

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
120517,lg:AOqpTOEuRDzBAl8r4_kyXgkmLFXU0n51uFDExjDgogL...,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Works fine.. I like the graphics and layout.. ...,5,4,1.0.1.0,2012-03-26 05:49:59,None,NaT
120516,lg:AOqpTOHtVCRQ280_md40DmafRCh73olr3srKIPr2UV6...,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,very convenient to use..,5,0,1.0.1.0,2012-05-08 03:32:34,None,NaT
120515,lg:AOqpTOFFSwUPBt4vUzDjb_A0tpfUPkQmXFc8YZmHUeC...,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Its working fine with my motorola droid razr. ...,5,1,1.0.1.0,2012-06-20 13:38:43,Thank you for giving us a 5-star rating! Keep ...,2018-02-23 10:26:01
120514,lg:AOqpTOEqa8V0KOvOM57V83v8t1BmodlxwEReZOjNfmP...,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,love the way you pay!,5,1,1.0.1.0,2012-07-02 14:10:54,None,NaT
120513,lg:AOqpTOEIFp10kTGbXQvLlgkWAEfIIPzEQ4sUTQG1TDx...,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,"I dont know about the others with low ratings,...",5,7,None,2012-08-10 01:51:20,None,NaT
...,...,...,...,...,...,...,...,...,...,...
120522,gp:AOqpTOEm2zsMX1ZfH4JQGKA_CtQLh6IMChVeiNkTn_d...,Randy Yenogacio,https://play-lh.googleusercontent.com/a/AATXAJ...,Easy transaction,5,0,None,2021-06-04 14:23:15,None,NaT
120521,gp:AOqpTOEfxumXvhm3fX_f6qXYph9Mi8zfgT6o-QmPzX6...,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Still hassle free for me..love gcash,5,0,5.12.0,2021-06-04 14:26:32,None,NaT
120520,gp:AOqpTOFfnA08GflWFxGwAqPRDg8-Ejy4_ORVThj6Qj0...,Bobbie Ocfemia,https://play-lh.googleusercontent.com/a-/AOh14...,great,5,0,5.40.0,2021-06-04 14:28:57,None,NaT
120519,gp:AOqpTOEMDuof3tzUX7rc2sZBqb17QFUg7VxRbFZlc_2...,ReyRose Tv,https://play-lh.googleusercontent.com/a-/AOh14...,Good for payment of bills,5,0,5.40.0,2021-06-04 14:30:14,None,NaT


In [27]:
# get review data from January 1, 2020 (12 AM) until June 3, 2021 (12 AM)
mask = (gps_df['at'] > '2020-01-01 00:00:00') & (gps_df['at'] < '2021-06-04 00:00:00')
five_star_df = gps_df.loc[mask]
five_star_df.sort_values(by=['at'])

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
82333,gp:AOqpTOE9vKOFEoLNN900Wwuid2T1RwonR1-FJDzw_iR...,Arnel Trono,https://play-lh.googleusercontent.com/a-/AOh14...,Nice,5,0,5.14.0,2020-01-01 00:54:16,None,NaT
82332,gp:AOqpTOFzPui58AHexouXqqE_fmFIL5xP-kvdUwv6S4Y...,Gerald Aguilar,https://play-lh.googleusercontent.com/a-/AOh14...,Awesome,5,0,None,2020-01-01 01:20:27,None,NaT
82331,gp:AOqpTOFjhUQfxEVrdMJqif7cENAfQ2KiniFY7nHKbJk...,rey mandalones,https://play-lh.googleusercontent.com/a-/AOh14...,faster,5,0,5.22.1,2020-01-01 02:22:00,None,NaT
82330,gp:AOqpTOETCFwyO01Q5qSMMLapG5qa0FV5yMkwZT3BkNo...,Hedie Survi,https://play-lh.googleusercontent.com/a-/AOh14...,Very useful,5,0,5.21.2,2020-01-01 02:31:30,None,NaT
82329,gp:AOqpTOEGrZxzbiBNH9v4HEixsh-8GRz0Fq3D-PMs4Xf...,Mortred Assasin,https://play-lh.googleusercontent.com/a/AATXAJ...,Good,5,0,None,2020-01-01 03:19:31,None,NaT
...,...,...,...,...,...,...,...,...,...,...
836,gp:AOqpTOFPthSWOyCdspYO4msEGh8m_1WvAxsSIwp5lbn...,Maener Cabuhat,https://play-lh.googleusercontent.com/a/AATXAJ...,excellent,5,0,5.40.0,2021-06-03 23:46:52,None,NaT
835,gp:AOqpTOH6Q1tJB1n05fQMvgqTMmqM-oyf7sa5bNepev5...,Karen Jane,https://play-lh.googleusercontent.com/a/AATXAJ...,excellent,5,0,5.40.0,2021-06-03 23:48:37,None,NaT
834,gp:AOqpTOG5j8qKnSyhjK_Zn4OlMF8MnRO9g1pHNqdV5sm...,Glena may Pike,https://play-lh.googleusercontent.com/a-/AOh14...,Super ganda nung app. daming tulong nito sakin...,5,0,5.40.0,2021-06-03 23:52:33,None,NaT
833,gp:AOqpTOH20galiBvWhl3m298Li5x8y3MpLqg0l91poCm...,rosemarie maranga,https://play-lh.googleusercontent.com/a-/AOh14...,ok,5,0,5.40.0,2021-06-03 23:53:29,None,NaT


## Export reviews dataframe as a csv file
---

In [28]:
five_star_df.to_csv('data/5_play_store_reviews.csv', index=False)

## Import 1-star Google Play Store reviews csv file
---

In [24]:
one_star_df = pd.read_csv('data/1_play_store_reviews.csv')

In [26]:
one_star_df.sort_values(by=['at'])

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
29187,gp:AOqpTOH039aUNokVUQY3HbgE9zPfHPph8U6k-H7Vbk1...,Nelson Virtus,https://play-lh.googleusercontent.com/a-/AOh14...,I always wanted to cash in for me to use it on...,1,0,5.24.0,2020-01-01 00:45:51,Hi! We are sorry to hear about your concern. P...,2020-01-01 13:31:55
29186,gp:AOqpTOH8Nm3rM0WhtcoY7SMRgYF_moiGgCm4tPtFrTL...,Clarrise Mijares,https://play-lh.googleusercontent.com/a/AATXAJ...,Pahelp nmn po dq maaus yung gcash mpin q dhil ...,1,0,5.20.2,2020-01-01 02:09:35,Hi! We are sorry to hear about your concern. P...,2020-01-01 13:31:52
29185,gp:AOqpTOFi19BO9dhYysEwieSZwR6piJ2VbE30pGaEbJT...,nica cunanan,https://play-lh.googleusercontent.com/a-/AOh14...,SINCE NOVEMBER UNTIL NOW I CANNOT CASH IN. BEE...,1,0,5.24.0,2020-01-01 02:47:23,Hi! We are sorry to hear about your concern. P...,2020-01-01 13:32:09
29184,gp:AOqpTOHS-CHmTAQxv9Qm3z_TK9cCMuKrtPDU2fNJDzK...,Muslimin Maranao,https://play-lh.googleusercontent.com/a-/AOh14...,Di maganda,1,0,5.22.1,2020-01-01 03:04:56,NaN,NaN
29183,gp:AOqpTOE7ZjpVBoiFBLcDAn0fFy-czijekaEU_LMa6vy...,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,POTEK NA GCASH TO...,1,0,5.24.0,2020-01-01 03:09:50,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
86,gp:AOqpTOHsVbq0GKGz3RtWPM8TG09sA0KT_pJqcR6aRRg...,Chris Nemenzo,https://play-lh.googleusercontent.com/a-/AOh14...,"Not safe. Beware, there are lots of verified s...",1,0,5.41.0,2021-06-03 23:24:17,NaN,NaN
1,gp:AOqpTOEmFMfv0vQwzWwmv8PZq7wGLjz_D-ATzJIHJmR...,Prodelyn Baulos,https://play-lh.googleusercontent.com/a/AATXAJ...,Good,1,0,5.39.1,2021-06-03 23:37:36,NaN,NaN
85,gp:AOqpTOEmFMfv0vQwzWwmv8PZq7wGLjz_D-ATzJIHJmR...,Prodelyn Baulos,https://play-lh.googleusercontent.com/a/AATXAJ...,Good,1,0,5.39.1,2021-06-03 23:37:36,NaN,NaN
84,gp:AOqpTOFEeyofoj8QCkehEIgF_NlZAy6V9MvNEzpKD4S...,Amador Jacla,https://play-lh.googleusercontent.com/a/AATXAJ...,Please update the app ASAP coz we r waiting th...,1,0,5.41.0,2021-06-03 23:44:31,NaN,NaN


## Combine 1-star and 5-star reviews dataframes
---

In [31]:
reviews_df = pd.concat([one_star_df, five_star_df])

In [32]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110690 entries, 0 to 82333
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   reviewId              110690 non-null  object
 1   userName              110690 non-null  object
 2   userImage             110690 non-null  object
 3   content               110688 non-null  object
 4   score                 110690 non-null  int64 
 5   thumbsUpCount         110690 non-null  int64 
 6   reviewCreatedVersion  84239 non-null   object
 7   at                    110690 non-null  object
 8   replyContent          2125 non-null    object
 9   repliedAt             2125 non-null    object
dtypes: int64(2), object(8)
memory usage: 9.3+ MB


## Data preprocessing
---

In [35]:
reviews_df = reviews_df.drop_duplicates()
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110606 entries, 0 to 82333
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   reviewId              110606 non-null  object
 1   userName              110606 non-null  object
 2   userImage             110606 non-null  object
 3   content               110604 non-null  object
 4   score                 110606 non-null  int64 
 5   thumbsUpCount         110606 non-null  int64 
 6   reviewCreatedVersion  84165 non-null   object
 7   at                    110606 non-null  object
 8   replyContent          2123 non-null    object
 9   repliedAt             2123 non-null    object
dtypes: int64(2), object(8)
memory usage: 9.3+ MB


## Export complete dataframe that contains all of GCash's 1-star and 5-star Google Play Store reviews as a csv file
---

In [6]:
reviews_df.to_csv('data/play_store_reviews.csv', index=False)